In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vector_store = FAISS.from_documents(docs, cache_embeddings)

retriver = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        (
            "human",
            "{question}",
        ),
    ]
)
memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = (
    {"context": retriver, "question": RunnablePassthrough(), "history": load_memory}
    | prompt
    | llm
)


In [2]:
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)


In [3]:
invoke_chain("Is Aaronson guilty?")

According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed. Therefore, he concludes that he invented it. This suggests that Aaronson is not actually guilty, but rather a victim of the Party's manipulation.


In [4]:
invoke_chain("What message did he write in the table?")

Winston traced "2+2=5" in the dust on the table.


In [5]:
invoke_chain("Who is Julia?")

Julia is a character who is Winston's love interest. She represents a form of rebellion against the oppressive regime of the Party. Winston has strong feelings for her, and their relationship is central to the narrative.


In [6]:
invoke_chain("What was the second question I asked?")

The second question you asked was, "What message did he write in the table?"
